In [12]:
from dotenv import load_dotenv
load_dotenv()
import os
from openai import OpenAI
from pinecone import Pinecone, ServerlessSpec



In [9]:
pc= Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
pc.create_index(
    name="rag",dimension=1536, metric="cosine", spec=ServerlessSpec(cloud="aws", region="us-east-1")
)


In [10]:
import json
data = json.load(open("reviews.json"))
data['reviews']

[{'professor': 'Dr. Sarah Johnson',
  'subject': 'Computer Science',
  'stars': 4,
  'review': "Dr. Johnson's lectures are engaging and she explains complex concepts clearly. Assignments can be challenging but rewarding."},
 {'professor': 'Prof. Michael Chen',
  'subject': 'Physics',
  'stars': 5,
  'review': 'Brilliant professor! Makes quantum mechanics understandable and exciting. His passion for the subject is contagious.'},
 {'professor': 'Dr. Emily Rodriguez',
  'subject': 'Psychology',
  'stars': 3,
  'review': 'Interesting course material, but lectures can be disorganized. Office hours are helpful for clarification.'},
 {'professor': 'Prof. David Kim',
  'subject': 'Economics',
  'stars': 4,
  'review': 'Prof. Kim brings real-world examples into his lectures, making macroeconomics relatable. Fair grader and approachable.'},
 {'professor': 'Dr. Lisa Patel',
  'subject': 'Biology',
  'stars': 5,
  'review': "Dr. Patel's enthusiasm for molecular biology is infectious. Lab sessions 

In [13]:
processed_data = []
client = OpenAI()

for review in data['reviews']:
    response = client.embeddings.create(
        input=review['review'],
        model="text-embedding-3-small",
    )
    embedding = response.data[0].embedding
    processed_data.append({
        "values":embedding,
        "id":review["professor"],
        "metadata":{
            "review":review["review"],
            "subject": review["subject"],
            "stars":review["stars"]
        }
    })


In [14]:
processed_data[0]

{'values': [-0.009603799,
  0.012425292,
  0.03578806,
  0.005147047,
  0.027879836,
  -0.007418985,
  0.00861192,
  0.028415987,
  0.00483876,
  0.028898522,
  0.037048012,
  0.014140975,
  -0.052006617,
  0.011533942,
  0.014060552,
  0.024314433,
  -0.044178817,
  -0.027008591,
  0.037262473,
  0.094576985,
  0.018054875,
  -0.0065577934,
  0.03326815,
  -0.009851769,
  -0.03991642,
  -0.057314508,
  -0.0028868364,
  -0.0077540795,
  0.031364817,
  -0.017089803,
  0.063212164,
  0.001988784,
  -0.023134902,
  -0.014476068,
  -0.016593864,
  0.058225967,
  0.008303634,
  0.03286604,
  0.017920837,
  0.020480957,
  -0.0063198763,
  0.019341636,
  -0.052810844,
  -0.018229123,
  0.010508554,
  0.009074351,
  0.007083891,
  -0.023201922,
  0.043991163,
  0.05972719,
  -0.024863988,
  0.024542298,
  0.01619175,
  -0.03565402,
  -0.048655674,
  -0.01825593,
  3.306437e-05,
  0.048226755,
  -0.043723088,
  -0.0029454778,
  0.025132064,
  -0.02017267,
  -0.0056228805,
  -2.7907063e-05,
  -0

In [15]:
index = pc.Index('rag')
index.upsert(
    vectors=processed_data,
    namespace="ns1"
)

{'upserted_count': 49}

In [16]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'ns1': {'vector_count': 49}},
 'total_vector_count': 49}